In [1]:
import pandas as pd
from database import DB_CONN
from pymongo import UpdateOne, ASCENDING

In [4]:
df = pd.read_excel('ipo_info.xlsx', header=0, dtype={'code':str})

In [5]:
df.head()

,code,name,issueprice,timeToMarket
0,000001,平安银行,40.0,1989-03-10
1,000002,万科A,1.0,1988-12-28
2,000004,国农科技,1.0,1989-12-23
3,000005,世纪星源,10.0,1990-02-28
4,000006,深振业A,10.0,1991-12-01


In [10]:
df = df.set_index('code')

In [12]:
codes = df.index.tolist()
# codes[:10]

In [15]:
df.issueprice['000001']

40.0

In [21]:
DB_CONN['basic'].find_one({'code':'300257'}, projection={'code':True,
                            'issueprice':True, 'timeToMarket':True,
                            '_id':False})

{'code': '300257', 'timeToMarket': '2011-08-19', 'issueprice': 63.0}

In [29]:
update_requests = []
for i,code in enumerate(codes):
    try:
        update_requests.append(
            UpdateOne(
            {'code':code},
            {'$set':{'issueprice':df.issueprice[code],
                    'timeToMarket':df.timeToMarket[code]}},
            upsert=True))
    except:
        print('code: %s, has problem' % code)

if len(update_requests) > 0:
    update_result = DB_CONN['basic'].bulk_write(update_requests, ordered=False)
    print('填充字段， 字段名: issueprice，数据集：%s，插入：%4d条，更新：%4d条' %
  ('basic', update_result.upserted_count, update_result.modified_count), flush=True)

填充字段， 字段名: issueprice，数据集：basic，插入：  93条，更新：3463条


In [ ]:
# for i,code in enumerate(codes):
#     result = DB_CONN['basic'].find_one({'code':code}, projection={'code':True,
#                             'issueprice':True, 'timeToMarket':True,
#                             '_id':False})
#     print(result)